<a href="https://colab.research.google.com/github/MihaiDogariu/Keysight-Deep-Learning-Fundamentals--v2-/blob/main/scripts/Unit_10_Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True

In [60]:
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip hymenoptera_data.zip


--2025-04-06 13:10:02--  https://download.pytorch.org/tutorial/hymenoptera_data.zip
Resolving download.pytorch.org (download.pytorch.org)... 18.172.170.8, 18.172.170.53, 18.172.170.43, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.172.170.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47286322 (45M) [application/zip]
Saving to: ‘hymenoptera_data.zip.1’

hymenoptera_data.zi 100%[===================>]  45.10M   240MB/s    in 0.2s    

2025-04-06 13:10:03 (240 MB/s) - ‘hymenoptera_data.zip.1’ saved [47286322/47286322]

Archive:  hymenoptera_data.zip
replace hymenoptera_data/train/ants/0013035.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: hymenoptera_data  hymenoptera_data.zip	hymenoptera_data.zip.1	sample_data


In [ ]:
train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [64]:
class HymenopteraDataset(torch.utils.data.Dataset):
    def __init__(self, dir_path, transform=None):
        self.dir_path = dir_path
        self.img_folder = torchvision.datasets.ImageFolder(dir_path, transform=transform)
        # torchvision.datasets.ImageFolder processes datasets that have the following structure:
        #
        # directory/
        # ├── class_x
        # │   ├── xxx.ext
        # │   ├── xxy.ext
        # │   └── ...
        # │   └── xxz.ext
        # └── class_y
        #     ├── 123.ext
        #     ├── nsdf3.ext
        #     └── ...
        #     └── asd932_.ext


    def __len__(self):
        return len(self.img_folder)

    def __getitem__(self, idx):
        return self.img_folder[idx]

In [74]:
train_dataset = HymenopteraDataset('hymenoptera_data/train')
val_dataset = HymenopteraDataset('hymenoptera_data/val')

In [76]:
len(val_dataset)

153

In [31]:
# prompt: pytorch script that loads a pre-trained vgg18 model and removes the last 3 layers

import torch
import torchvision.models as models

# Load the pre-trained VGG18 model
vgg19 = models.vgg19(pretrained=True)

# Remove the last three layers
vgg19_modified = torch.nn.Sequential(*list(vgg19.children())[:-1])

In [38]:
list(vgg19.classifier.children())[:-4]

[Linear(in_features=25088, out_features=4096, bias=True),
 ReLU(inplace=True),
 Dropout(p=0.5, inplace=False)]

In [40]:
vgg19_modified =  torch.nn.Sequential(*list(models.vgg19(pretrained=True).children()))
print(vgg19_modified)
vgg19_modified.classifier = torch.nn.Sequential(*list(vgg19.classifier.children())[:-1])
print(vgg19_modified)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd